<a href="https://colab.research.google.com/github/AlexXPZhu/XMUM-FYP-Code/blob/main/MergeFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 这个用来把三个文件融合，包括 sqlv1 sqlv2 和 modified data

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/FYP/sqliv2.csv'
sqli2 = pd.read_csv(file_path, encoding='utf-16')


display(sqli2.head(10))

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1
5,select * from users where id = '1' or @ @1 = ...,1
6,"select * from users where id = 1 or 1#"" ( un...",1
7,' select name from syscolumns where id = ( ...,1
8,select * from users where id = 1 +$+ or 1 = 1...,1
9,"1; ( load_file ( char ( 47,101,116,99,47,112,9...",1


In [ ]:
sqli = pd.read_csv('/content/drive/MyDrive/FYP/sqli.csv', encoding='utf-16')
display(sqli.head(10))



,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1
5,?,1
6,' and 1 = 0 ) union all,1
7,? or 1 = 1 --,1
8,x' and userid is NULL; --,1
9,x' and email is NULL; --,1


In [ ]:
Modified = pd.read_csv('/content/drive/MyDrive/FYP/Modified_SQL_Dataset.csv')
display(Modified.head(10))



,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = ...,1
6,select * from users where id = 1 +$+ or 1 =...,1
7,"1; ( load_file ( char ( 47,101,116,99,47...",1
8,select * from users where id = '1' or ||/1 ...,1
9,select * from users where id = '1' or \.<\ ...,1


In [ ]:
merged = pd.concat([sqli, sqli2, Modified], ignore_index=True)
merged['Sentence'] = merged['Sentence'].fillna(merged['Query'])
merged.drop(columns=['Query'], inplace=True)
display(merged.head(10))
merged.info()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1
5,?,1
6,' and 1 = 0 ) union all,1
7,? or 1 = 1 --,1
8,x' and userid is NULL; --,1
9,x' and email is NULL; --,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68880 entries, 0 to 68879
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  68863 non-null  object
 1   Label     68880 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [ ]:
# 对于融合后的数据进行去重，去除空内容
merged = merged.drop_duplicates(subset=['Sentence'])
merged = merged.dropna(subset=['Sentence'])
merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56897 entries, 0 to 68879
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  56897 non-null  object
 1   Label     56897 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [ ]:
output_file_path = '/content/drive/MyDrive/FYP/merged_data.csv'
merged.to_csv(output_file_path, index=False)
print(f'Merged data saved to {output_file_path}')

Merged data saved to /content/drive/MyDrive/FYP/merged_data.csv
